In [ ]:
# import the function
import sys
sys.path.append(os.path.abspath('..'))

import functs as fn
import numpy as np

ModuleNotFoundError: No module named 'functs'

In [ ]:
# set dimension
d = 10

# define external field and couplings
h = np.zeros(d)
j = np.random.normal(0, 0.2, size=(d, d))
j = (j + j.T) / 2
np.fill_diagonal(j, 0)

# generate samples
samples = fn.generate_data_ising(
    n_data=50,     # number of configurations to sample
    h=h,
    j=j,
    n_steps=100,
    burn_in=500,
    beta=1.0
)

# check shape of result
print("generated data shape:", samples.shape)